In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, isnan, count
from src.utils.s3_manager import S3Manager
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os 
import nltk


In [2]:
data_dir = "/Users/ilan/big-data-airflow-project/data"

In [3]:
spark = SparkSession.builder \
    .appName("EDA with Spark") \
    .getOrCreate()

24/05/17 10:04:26 WARN Utils: Your hostname, Ordinateur-portable-de-Ilan.local resolves to a loopback address: 127.0.0.1; using 10.3.155.210 instead (on interface en0)
24/05/17 10:04:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/17 10:04:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.parquet(data_dir+"/allocine_movies.parquet", header=True, inferSchema=True)

In [5]:
df.show()

+--------------------+--------+------+---------------+------------------+--------------------+------------+----------------+--------------------+
|               Title|Duration| Genre|   Release Date|          Director|              Actors|Press Rating|Spectator Rating|            Synopsis|
+--------------------+--------+------+---------------+------------------+--------------------+------------+----------------+--------------------+
|The Dark Knight, ...|2h 32min|Action|            N/A| Christopher Nolan|Christian Bale, P...|         4,0|             4,5|Batman entreprend...|
|           Gladiator|2h 35min|Action|            N/A|      Ridley Scott|Russell Crowe, Jo...|         4,3|             4,5|Le général romain...|
|Spider-Man : Acro...|2h 21min|Action|            N/A|Joaquim Dos Santos|Stéphane Bak, Sha...|         4,1|             4,4|Après avoir retro...|
|Spider-Man : New ...|1h 57min|Action|            N/A|   Bob Persichetti|        Jake Johnson|         4,2|             4,4|

In [6]:
print("The shape of the allocine dataset is ", (df.count(), len(df.columns)))

The shape of the allocine dataset is  (3379, 9)


In [7]:
df.describe().show()

24/05/17 10:06:35 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+--------+-------+------------+--------------+--------------------+------------+----------------+--------------------+
|summary|               Title|Duration|  Genre|Release Date|      Director|              Actors|Press Rating|Spectator Rating|            Synopsis|
+-------+--------------------+--------+-------+------------+--------------+--------------------+------------+----------------+--------------------+
|  count|                3379|    3378|   3379|        3379|          3376|                3372|        3374|            3379|                3376|
|   mean|               796.0|    NULL|   NULL|        NULL|          NULL|                NULL|        NULL|            NULL|                NULL|
| stddev|   874.0533164515766|    NULL|   NULL|        NULL|          NULL|                NULL|        NULL|            NULL|                NULL|
|    min|                 '71|0h 06min| Action| 1 août 2001|  Costa-Gavras| Anémone, Josiane...|         1,3|   

# Renaming columns to match the netflix dataset

In [11]:
df = df.withColumnRenamed("Title", "title")
df = df.withColumnRenamed("Duration", "duration")
df = df.withColumnRenamed("Synopsis", "description")
df = df.withColumnRenamed("Release Date", "release_year")


# Dealing with missing values

In [12]:
missing_values = df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns])
missing_values.show()

+-----+--------+-----+------------+--------+------+------------+----------------+-----------+
|title|duration|Genre|release_year|Director|Actors|Press Rating|Spectator Rating|description|
+-----+--------+-----+------------+--------+------+------------+----------------+-----------+
|    0|       1|    0|           0|       3|     7|           5|               0|          3|
+-----+--------+-----+------------+--------+------+------------+----------------+-----------+
